In [5]:
with open(r"F:\projects\PlayGround\gsoft\data\tiny_shakespeare.txt")as f:
    text = f.read()

In [6]:
print(len(text))

1115394


In [9]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [11]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

In [13]:
print(encode("Hello World!!"))
print(decode(encode("Hello World!!")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2, 2]
Hello World!!


In [14]:
import torch

In [16]:
data =  torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [18]:
#split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [19]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [25]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [35]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix  = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [39]:
xb, yb = get_batch('train')
print(xb.shape)
print(xb,"----", yb)

torch.Size([4, 8])
tensor([[43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39]]) ---- tensor([[ 1, 51, 39, 63,  1, 40, 43,  1],
        [46, 43,  1, 43, 39, 56, 57, 10],
        [58, 47, 53, 52, 12,  1, 37, 53],
        [56, 43,  1, 21,  1, 41, 39, 51]])


In [41]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when context is {context} the target is {target}")
    break

when context is tensor([43]) the target is 1
when context is tensor([43,  1]) the target is 51
when context is tensor([43,  1, 51]) the target is 39
when context is tensor([43,  1, 51, 39]) the target is 63
when context is tensor([43,  1, 51, 39, 63]) the target is 1
when context is tensor([43,  1, 51, 39, 63,  1]) the target is 40
when context is tensor([43,  1, 51, 39, 63,  1, 40]) the target is 43
when context is tensor([43,  1, 51, 39, 63,  1, 40, 43]) the target is 1


In [42]:
print(xb)

tensor([[43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39]])


In [43]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [49]:
class BigramLanguage(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        
        logits = self.token_embedding_table(idx) # (B,T,C) batch, time, channel [4, 8, 65]
        
        
        return logits

In [50]:
m = BigramLanguage(vocab_size)
out = m(xb,yb)
print(out.shape)

torch.Size([4, 8, 65])


In [63]:
class BigramLanguage(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        logits = self.token_embedding_table(idx) # (B,T,C) batch, time, channel [4, 8, 65]
        
        if targets == None:
            loss = None
        else:
            
            B, T, C = logits.shape
            
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
        
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        
        return idx

In [64]:
m = BigramLanguage(vocab_size)
logits, loss2 = m(xb,yb)
print(loss2)

tensor(4.3543, grad_fn=<NllLossBackward0>)


In [65]:
idx = torch.zeros((1,1), dtype=torch.long)
idx

tensor([[0]])

In [66]:
decode(m.generate(idx=idx, max_new_tokens=100)[0].tolist())

'\nKL3fH-?lc;KmWHx -xAILJ;;EABcafTJ $PDJ:A?ppHXpFz:kXanLu,Y-Nu tjRE,dTIAadZ3!&WBbaDpcUW mACYRXq c;-GbRF'

In [67]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [79]:
batch_size = 32
for steps in range(10000):
    
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.2799112796783447


In [80]:
print(decode(m.generate(idx=idx, max_new_tokens=100)[0].tolist()))



ONENGoknde's as IUnd bl nothowe m hith ineld
T:
Fit bengensk s,
T:
LOFilot ' thin! kinor curcoul am


In [82]:
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [83]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [88]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b 
print(a)
print("---------")
print(b)
print("---------")
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---------
tensor([[5., 3.],
        [1., 1.],
        [4., 0.]])
---------
tensor([[5.0000, 3.0000],
        [3.0000, 2.0000],
        [3.3333, 1.3333]])


In [91]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow,xbow2)

True

In [107]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril ==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow,xbow2)

In [108]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])